In [ ]:
import tensorflow as tf
import numpy as np
import input_data

In [ ]:
learning_rate = 0.001 # 0.5 진동한다.
training_epochs = 15
batch_size = 100
display_step = 1

In [ ]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=True)

## dummy initialize

In [ ]:
X = tf.placeholder('float', [None, 784])
Y = tf.placeholder('float', [None, 10])

W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))

L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
#y_ = tf.nn.softmax(tf.matmul(L2, W3) + b3)
y_ = tf.add(tf.matmul(L2, W3), b3) # No need to use softmax here. because softmax is called in cost function.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, Y))
#cost = tf.reduce_mean(- tf.reduce_sum(Y * tf.log(y_), 1))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## xavier initialize

In [ ]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev = stddev)
    

In [ ]:
X = tf.placeholder('float', [None, 784])
Y = tf.placeholder('float', [None, 10])

W1 = tf.get_variable("W1", shape=[784, 256], initializer = xavier_init(784, 256))
b1 = tf.Variable(tf.random_normal([256]))
W2 = tf.get_variable("W2", shape=[256, 256], initializer = xavier_init(256, 256))
b2 = tf.Variable(tf.random_normal([256]))
W3 = tf.get_variable("W3", shape=[256, 10], initializer = xavier_init(256, 10))
b3 = tf.Variable(tf.random_normal([10]))

L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
#y_ = tf.nn.softmax(tf.matmul(L2, W3) + b3)
y_ = tf.add(tf.matmul(L2, W3), b3) # No need to use softmax here. because softmax is called in cost function.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, Y))
#cost = tf.reduce_mean(- tf.reduce_sum(Y * tf.log(y_), 1))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        cost_val, _ = sess.run([cost, optimizer], feed_dict = {X:batch_xs, Y:batch_ys})
        avg_cost += cost_val / total_batch
    if (epoch % display_step == 0):
        print("Epoch:", '%04d' %(epoch + 1), "cost=", "{:.9f}".format(avg_cost))
print("Optimization Finished")

In [ ]:
sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels})

In [ ]:
sess.close()